# **1 Import and installations**

In [17]:
!pip install bert-tensorflow


     |████████████████████████████████| 71kB 3.5MB/s 


In [19]:
pip install pytorch-pretrained-bert


     |████████████████████████████████| 133kB 4.9MB/s 


In [5]:
pip install seqeval


     |████████████████████████████████| 51kB 2.9MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16172 sha256=ef098a01bed18323b8bc32e3f02601699f7464ff9a11d7c34eeee7bc488456fc
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval


In [ ]:
pip install tensorflow-hub


In [ ]:
pip install tensorflow


In [ ]:
!pip install pytorch-transformers


In [ ]:
pip install tensorflow-gpu

In [42]:
import bert
from bert import tokenization
from tqdm import tqdm, trange
import tensorflow as tf
from torch.autograd import Variable
##
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig , BertModel
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam
import itertools
from seqeval.metrics import f1_score
# from bert import run_classifier
# from bert import optimization
# from bert import tokenization
tf.config.experimental.list_physical_devices('GPU')
#group in sentences
import pandas as pd
import numpy as np
import tensorflow_hub as hub
from sklearn.metrics import accuracy_score


# **2 Constants**

In [38]:
MAX_LEN = 75
BATCH_SIZE = 32
LABEL_LIST = ['B-LOC', 'I-PER', 'B-MISC', 'I-ORG', 'I-MISC','B-ORG', 'O','I-LOC']
EPOCHS = 3
EN = 'en'


# **3 Functions**
> data pre-processing 




In [10]:


def dataPreprocessing(path):
  '''
  function which takes an input file for bert and process it.
  @param path - path to the dataset
  returns - 
  sentences create senctence from data,
  labels - label of each sentence 
    
    O - Outside of a named entity
    B-MIS -  Beginning of a miscellaneous entity right after another miscellaneous entity
    I-MIS  - Miscellaneous entity 
    B-PER - Beginning of a person’s name right after another person’s name
    I-PER  - Person’s name 
    B-ORG - Beginning of an organisation right after another organisation 
    I-ORG - Organisation 
    B-LOC - Beginning of a location right after another location
    I-LOC - Location
    location (LOC), organizations (ORG), person (PER) and Miscellaneous (MISC).
  tags_vals - all optional tags. ['I-PER', 'I-MISC', 'B-ORG', 'B-MISC', 'I-ORG', 'B-LOC', 'O', 'I-LOC']
  tags2idx - dictionary with value for each tag - example :
    {I-PER: 1, 'I-MISC': 2, 'B-ORG': 3, 'B-MISC': 4, 'I-ORG': 5, 'B-LOC': 6, 'O': 7, 'I-LOC': 8}
  tag_count - count appreances of each tag
  data - data frame with entire data 
    [index', 'Word', 'POS', 'Sentence#', 'Tag', 'fullSentence']
  getter.sentences - sentence iterator
  
  '''
  data = pd.read_csv(path,sep = " ").fillna(0)
  data.columns = ['Word', 'POS','Sentence#','Tag']
  #remove O - not intersting
  data = data[data["POS"] != 'O']
  data = data.reset_index()
  #get all end of sentence indexs
  sentence_sep = np.insert(data.index[data['POS'] == '.']+1,0,0)
  # length of sentences
  ranges = np.insert(np.asarray(sentence_sep[1:] - sentence_sep[:-1]),len(sentence_sep)-1,data.shape[0]-sentence_sep[-1])# for i in range(len(ranges)):

  # succsessive id for number of sentence we are at (0,0,0,1,1,1,1,2,2,2,2,..)
  sentenceNum = list(itertools.chain.from_iterable(itertools.repeat(x,r) for x,r in zip(range(len(ranges)),ranges)))
  # inner enumeration of each sentence (0,1,2,0,1,2,3,0,1,2,3...)
  innerPos = list(itertools.chain.from_iterable(range(r) for x,r in zip(range(len(ranges)),ranges)))
  data["POS"] = innerPos
  data["Sentence#"] = sentenceNum


  #iterator of sentences
  getter = SentenceGetter(data)

  sentences = [" ".join([str(s[0]) for s in sent]) for sent in getter.sentences]
  labels = [[s[2] for s in sent] for sent in getter.sentences]
  
  data["fullSentence"] = list(itertools.chain.from_iterable(itertools.repeat(x,r) for x,r in zip(sentences,ranges)))
  data["fullSentenceTag"] = list(itertools.chain.from_iterable(itertools.repeat(x,r) for x,r in zip(labels,ranges)))

  segments = [["0" for s in sent] for sent in sentences]

  # labels = [lab+["[SE for lab in labels]
  # sentences = ["[CLS]"+sen+"[SEP]" for sen in sentences]
  #contains all tages
  tags_vals = list(set(data["Tag"].values))
  # contains a dictionary of the tags - each enumerated 
  tag2idx = {t: i+1 for i, t in enumerate(tags_vals)}
  tag_count = {}
  for label in tags_vals:
      tag_count[label] = len(data.index[data['Tag'] == label])
  
  return sentences,labels,tags_vals,tag2idx,tag_count,data,getter.sentences


class SentenceGetter(object):
  '''
  generate a sentence iterator
  each return yields 3 different lists : word,POS,tag which 
  correspond to each other
  '''

  def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence#").apply(agg_func)
        self.sentences = [s for s in self.grouped]

  def get_next(self):
        try:
            s = self.grouped[self.n_sent]
            self.n_sent += 1
            return s
        except:
            return None



Initialize model and set the set model hyperparameters

In [11]:
def get_hyperparameters(model, ff):

    # ff: full_finetuning
    if ff:
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "gamma", "beta"]
        optimizer_grouped_parameters = [
            {
                "params": [
                    p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
                ],
                "weight_decay_rate": 0.01,
            },
            {
                "params": [
                    p for n, p in param_optimizer if any(nd in n for nd in no_decay)
                ],
                "weight_decay_rate": 0.0,
            },
        ]
    else:
        param_optimizer = list(model.classifier.named_parameters())
        optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

    return optimizer_grouped_parameters



train model


In [45]:
def train_and_save_model(
    model,
    tokenizer,
    optimizer,
    idx2tag,
    tag2idx,
    this_run,
    max_grad_norm,
    device,
    train_dataloader,
    valid_dataloader,
):

    pad_tok, sep_tok, cls_tok, o_lab = get_special_tokens(tokenizer, tag2idx)
    verbose = True
    epochs = EPOCHS

    epoch = 0
    for _ in trange(epochs, desc="Epoch"):
        epoch += 1

        # Training loop
        print("Starting training loop.")
        model.train()
        tr_loss, tr_accuracy = 0, 0
        nb_tr_examples, nb_tr_steps = 0, 0
        tr_preds, tr_labels = [], []

        for step, batch in enumerate(train_dataloader):

            # Add batch to gpu
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch

            # Forward pass
            outputs = model(
                b_input_ids,
                token_type_ids=None,
                attention_mask=b_input_mask,
                labels=b_labels,
            )
            loss, tr_logits = outputs[:2]

            # Backward pass
            loss.backward()

            # Compute train loss
            tr_loss += loss.item()
            nb_tr_examples += b_input_ids.size(0)
            nb_tr_steps += 1

            # Subset out unwanted predictions on CLS/PAD/SEP tokens
            preds_mask = (
                (b_input_ids != cls_tok)
                & (b_input_ids != pad_tok)
                & (b_input_ids != sep_tok)
            )

            tr_logits = tr_logits.detach().cpu().numpy()
            tr_label_ids = torch.masked_select(b_labels, (preds_mask == 1))
            tr_batch_preds = np.argmax(tr_logits[preds_mask.squeeze()], axis=1)
            tr_batch_labels = tr_label_ids.to("cpu").numpy()
            tr_preds.extend(tr_batch_preds)
            tr_labels.extend(tr_batch_labels)

            # Compute training accuracy
            tmp_tr_accuracy = accuracy_score(tr_batch_labels, tr_batch_preds)
            tr_accuracy += tmp_tr_accuracy

            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(
                parameters=model.parameters(), max_norm=max_grad_norm
            )

            # Update parameters
            optimizer.step()
            model.zero_grad()

        tr_loss = tr_loss / nb_tr_steps
        tr_accuracy = tr_accuracy / nb_tr_steps

        # Print training loss and accuracy per epoch
        print(f"Train loss: {tr_loss}")
        print(f"Train accuracy: {tr_accuracy}")

        # Validation loop
        print("Starting validation loop.")

        model.eval()
        eval_loss, eval_accuracy = 0, 0
        nb_eval_steps, nb_eval_examples = 0, 0
        predictions, true_labels = [], []

        for batch in valid_dataloader:

            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch

            with torch.no_grad():
                outputs = model(
                    b_input_ids,
                    token_type_ids=None,
                    attention_mask=b_input_mask,
                    labels=b_labels,
                )
                tmp_eval_loss, logits = outputs[:2]

            # Subset out unwanted predictions on CLS/PAD/SEP tokens
            preds_mask = (
                (b_input_ids != cls_tok)
                & (b_input_ids != pad_tok)
                & (b_input_ids != sep_tok)
            )

            logits = logits.detach().cpu().numpy()
            label_ids = torch.masked_select(b_labels, (preds_mask == 1))
            val_batch_preds = np.argmax(logits[preds_mask.squeeze()], axis=1)
            val_batch_labels = label_ids.to("cpu").numpy()
            predictions.extend(val_batch_preds)
            true_labels.extend(val_batch_labels)

            tmp_eval_accuracy = flat_accuracy(val_batch_labels, val_batch_preds)

            eval_loss += tmp_eval_loss.mean().item()
            eval_accuracy += tmp_eval_accuracy

            nb_eval_examples += b_input_ids.size(0)
            nb_eval_steps += 1

        # Evaluate loss, acc, conf. matrix, and class. report on devset
        pred_tags = [idx2tag[i] for i in predictions]
        valid_tags = [idx2tag[i] for i in true_labels]
        cl_report = classification_report(valid_tags, pred_tags)
        conf_mat = annot_confusion_matrix(valid_tags, pred_tags)
        eval_loss = eval_loss / nb_eval_steps
        eval_accuracy = eval_accuracy / nb_eval_steps

        # Report metrics
        print(f"Validation loss: {eval_loss}")
        print(f"Validation Accuracy: {eval_accuracy}")
        print(f"Classification Report:\n {cl_report}")
        if verbose:
            print(f"Confusion Matrix:\n {conf_mat}")

        # Save model and optimizer state_dict following every epoch
        save_path = f"../models/{this_run}/train_checkpoint_epoch_{epoch}.tar"
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "train_loss": tr_loss,
                "train_acc": tr_accuracy,
                "eval_loss": eval_loss,
                "eval_acc": eval_accuracy,
                "classification_report": cl_report,
                "confusion_matrix": conf_mat,
            },
            save_path,
        )
        print(f"Checkpoint saved to {save_path}.")

In [13]:
def get_special_tokens(tokenizer, tag2idx):

    pad_tok = tokenizer.vocab["[PAD]"]
    sep_tok = tokenizer.vocab["[SEP]"]
    cls_tok = tokenizer.vocab["[CLS]"]
    o_lab = tag2idx["O"]

    return pad_tok, sep_tok, cls_tok, o_lab

mount collab

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **4 Preprocess data**

## 4.1 Get input file and extract sentences, labels

In [47]:
train_sentences,train_labels,tags_vals,tag2idx,tag_count,train_data,sentences = dataPreprocessing("sample_data/eng.train.csv")
test_sentences,test_labels,_,_,_,test_data,_ = dataPreprocessing("sample_data/eng.testa.csv")

train_sentences= train_sentences[:1000]
train_labels = train_labels[:1000]
test_sentences = test_sentences[:200]
test_labels = test_labels[:200]
print(tag_count)


{'B-ORG': 24, 'I-PER': 11128, 'B-LOC': 11, 'B-MISC': 37, 'I-MISC': 4556, 'I-ORG': 10001, 'I-LOC': 8286, 'O': 168345}


In [54]:
train_data.head(10)

,index,Word,POS,Sentence#,Tag,fullSentence,fullSentenceTag
0,0,EU,0,0,I-ORG,EU rejects German call to boycott British lamb .,"[I-ORG, O, I-MISC, O, O, O, I-MISC, O, O]"
1,1,rejects,1,0,O,EU rejects German call to boycott British lamb .,"[I-ORG, O, I-MISC, O, O, O, I-MISC, O, O]"
2,2,German,2,0,I-MISC,EU rejects German call to boycott British lamb .,"[I-ORG, O, I-MISC, O, O, O, I-MISC, O, O]"
3,3,call,3,0,O,EU rejects German call to boycott British lamb .,"[I-ORG, O, I-MISC, O, O, O, I-MISC, O, O]"
4,4,to,4,0,O,EU rejects German call to boycott British lamb .,"[I-ORG, O, I-MISC, O, O, O, I-MISC, O, O]"
5,5,boycott,5,0,O,EU rejects German call to boycott British lamb .,"[I-ORG, O, I-MISC, O, O, O, I-MISC, O, O]"
6,6,British,6,0,I-MISC,EU rejects German call to boycott British lamb .,"[I-ORG, O, I-MISC, O, O, O, I-MISC, O, O]"
7,7,lamb,7,0,O,EU rejects German call to boycott British lamb .,"[I-ORG, O, I-MISC, O, O, O, I-MISC, O, O]"
8,8,.,8,0,O,EU rejects German call to boycott British lamb .,"[I-ORG, O, I-MISC, O, O, O, I-MISC, O, O]"
9,9,Peter,0,1,I-PER,Peter Blackburn BRUSSELS 1996-08-22 The Europe...,"[I-PER, I-PER, I-LOC, O, O, I-ORG, I-ORG, O, O..."


In [29]:
train_sentences[0]

'EU rejects German call to boycott British lamb .'

In [30]:
train_labels[0]

['I-ORG', 'O', 'I-MISC', 'O', 'O', 'O', 'I-MISC', 'O', 'O']

## 4.2 Embed the data
Adding pos of word as part of the sentence

In [48]:
#tokenize the senctence to a form bert will understand - 
#uncased: it does not make a difference between english and English.
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
# tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
#tokenize each sentence for all its words ##ing
train_data_em  = [str(train_data["Word"].iloc[i])+" "+str(train_data["POS"].iloc[i])+" "
                 +str(train_data["fullSentence"].iloc[i]) for i in range(train_data.shape[0])]

train_data_em[0]






100%|██████████| 231508/231508 [00:00<00:00, 3424444.48B/s]


'EU 0 EU rejects German call to boycott British lamb .'

In [49]:
train_tag_em  = [str(train_data["Tag"].iloc[i])+" "+'O'+" "
                 +" ".join(train_data["fullSentenceTag"].iloc[i]) for i in range(train_data.shape[0])]


In [26]:
train_tag_em[0]

'I-ORG O I-ORG O I-MISC O O O I-MISC O O'

In [50]:
test_data_em = [str(test_data["Word"].iloc[i])+" "+str(test_data["POS"].iloc[i])+" "
                 +str(test_data["fullSentence"].iloc[i]) for i in range(test_data.shape[0])]

test_data_em[0]

'CRICKET 0 CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTER INNINGS VICTORY .'

In [51]:
test_tag_em  = [str(test_data["Tag"].iloc[i])+" "+'O'+" "
                 +" ".join(test_data["fullSentenceTag"].iloc[i]) for i in range(test_data.shape[0])]


In [29]:
test_tag_em[0]

# [str(sen).split(" ") for sen in train_tag_em][0]
# str(train_tag_em[0])

'O O O O I-ORG O O O O O O O O'

## 4.3 Tokenize each sentence

In [52]:
'''
lower case all and insert to an array
example (2 is the place of german in the sentence)
['german', '2', 'eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.']
German 2 EU rejects German call to boycott British lamb .
'''
train_tokens = [tokenizer.tokenize(sent) for sent in train_data_em]
test_tokens = [tokenizer.tokenize(sent) for sent in test_data_em]

train_tag_tokens = [sen.split(" ") for sen in train_tag_em]
test_tag_tokens = [sen.split(" ") for sen in test_tag_em]

print(train_tokens[0])
print(test_tokens[0])

['eu', '0', 'eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.']
['cricket', '0', 'cricket', '-', 'leicestershire', 'take', 'over', 'at', 'top', 'after', 'innings', 'victory', '.']


## 4.4 Convert to wordpiece ids + extract MAX_LEN sentences (padd)

In [53]:
'''
convert the words to word2vec encode
example array([ 7327,  1014,  7327, 19164,  2446,  2655,  2000, 17757,  2329,
       12559,  1012,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0])
'''
train_tokens_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt[:512]) for txt in train_tokens],
                           maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

test_tokens_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt[:512]) for txt in test_tokens],
                           maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")


In [54]:
# test_labels = [str(var) for var in test_data["Tag"]]
# train_labels = [str(var) for var in train_data["Tag"]]
print(f'number of train sentences {len(train_tokens_ids)}  number of train labels {len(train_tag_tokens)}')
print(f'number of test sentences {len(test_tokens_ids)}  number of test labels {len(test_tag_tokens)}')


train_tag_ids = pad_sequences([[tag2idx.get(l) for l in lab] for lab in train_tag_tokens],
                     maxlen=MAX_LEN, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")

test_tag_ids = pad_sequences([[tag2idx.get(l) for l in lab] for lab in test_tag_tokens],
                     maxlen=MAX_LEN, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")






number of train sentences 202388  number of train labels 202388
number of test sentences 50937  number of test labels 50937


## 4.5 Convert data to tensors and load tensors into DataLoaders 

In [55]:
train_attn_masks = [[float(i > 0) for i in ii] for ii in train_tokens_ids]
test_attn_masks = [[float(i > 0) for i in ii] for ii in test_tokens_ids]


In [56]:
#create a sample of words which we will MASK
tr_inputs = torch.tensor(train_tokens_ids)
val_inputs = torch.tensor(test_tokens_ids)
tr_tags = torch.tensor(train_tag_ids)
val_tags = torch.tensor(test_tag_ids)
tr_masks = torch.tensor(train_attn_masks)
val_masks = torch.tensor(test_attn_masks)

train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(
    train_data, sampler=train_sampler, batch_size=BATCH_SIZE
)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(
    valid_data, sampler=valid_sampler, batch_size=BATCH_SIZE
)

In [142]:
{tag2idx[key] : key for key in tag2idx.keys()}

{1: 'B-LOC',
 2: 'I-PER',
 3: 'B-MISC',
 4: 'I-ORG',
 5: 'I-MISC',
 6: 'B-ORG',
 7: 'O',
 8: 'I-LOC'}

## Fine tune bert

In [ ]:
import argparse
import csv
import datetime as dt
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import os
import pandas as pd
from pytorch_transformers import BertTokenizer, BertForTokenClassification, BertConfig
import re
from seqeval.metrics import classification_report
from sklearn.metrics import confusion_matrix
import spacy
from spacy.gold import Doc, biluo_tags_from_offsets
import subprocess
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm, trange
import yaml



# Constants
label_types = LABEL_LIST
MAX_LEN = MAX_LEN
BATCH_SIZE = BATCH_SIZE
EPOCHS = EPOCHS
MODEL = 'bert-base-uncased'
THIS_RUN = dt.datetime.now().strftime("%m.%d.%Y, %H.%M.%S")
MAX_GRAD_NORM = 1.0
NUM_LABELS = len(label_types)+1
FULL_FINETUNING = True
idx2tag = {tag2idx[key] : key for key in tag2idx.keys()}
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
print("Loaded training and validation data into DataLoaders.")

# Initialize model
model = BertForTokenClassification.from_pretrained(MODEL, num_labels=NUM_LABELS)
model.to(device)
print(f"Initialized model and moved it to {device}.")

# Set hyperparameters (optimizer, weight decay, learning rate)
optimizer_grouped_parameters = get_hyperparameters(model, FULL_FINETUNING)
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)
print("Initialized optimizer and set hyperparameters.")

    # Fine-tune model and save checkpoint every epoch
train_and_save_model(
    model,
    tokenizer,
    optimizer,
    idx2tag,
    tag2idx,
    THIS_RUN,
    MAX_GRAD_NORM,
    device,
    train_dataloader,
    valid_dataloader)


Loaded training and validation data into DataLoaders.






Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Initialized model and moved it to cpu.
Initialized optimizer and set hyperparameters.
Starting training loop.
